In [11]:
with open('../data/reviews.json') as file:
    head = [next(file) for _ in range(50)]
    
head[0:2]

['{"votes": {"funny": 0, "useful": 2, "cool": 1}, "user_id": "Xqd0DzHaiyRqVH3WRG7hzg", "review_id": "15SdjuK7DmYqUAj6rjGowg", "stars": 5, "date": "2007-05-17", "text": "dr. goldberg offers everything i look for in a general practitioner.  he\'s nice and easy to talk to without being patronizing; he\'s always on time in seeing his patients; he\'s affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i\'m sitting here trying to think of any complaints i have about him, but i\'m really drawing a blank.", "type": "review", "business_id": "vcNAWiLM4dR7D2nwwJ7nCA"}\n',
 '{"votes": {"funny": 0, "useful": 2, "cool": 0}, "user_id": "H1kH6QZV7Le4zqTRNxoZow", "review_id": "RF6UnRTtG7tWMcrO2GEoAg", "stars": 2, "date": "2010-03-22", "text": "Unfortunately, the frustration of being Dr. Goldberg\'s p

In [14]:
import json

parsed = [json.loads(review) for review in head]

parsed[0]

{'business_id': 'vcNAWiLM4dR7D2nwwJ7nCA',
 'date': '2007-05-17',
 'review_id': '15SdjuK7DmYqUAj6rjGowg',
 'stars': 5,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
 'type': 'review',
 'user_id': 'Xqd0DzHaiyRqVH3WRG7hzg',
 'votes': {'cool': 1, 'funny': 0, 'useful': 2}}

In [18]:
class Example():
    def __init__(self, review, votes):
        self.review = review
        self.votes = votes
        
    def __str__(self):
        return "Review: " + self.review[0:100]
    
    __repr__ = __str__
    
e = Example(parsed[0]['text'], parsed[0]['votes'])

e

Review: dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to 

In [20]:
import pybrain

ImportError: No module named 'structure'